# Implement ปัญหา 8-Puzzle

```
ไพรสันต์ ผดุงเวียง 2025

```



In [1]:
from typing import  List, Optional, Tuple
from kamgon import *

# State คือ Tuple ของตัวเลข 9 ตัว, Action คือ string 'UP', 'DOWN', 'LEFT', 'RIGHT'
PuzzleState = Tuple[int, ...]
PuzzleAction = str

class EightPuzzleProblem(Problem[PuzzleState, PuzzleAction]):
    """
    คลาสสำหรับปัญหา 8-Puzzle ซึ่งเป็นปัญหาการเลื่อนตัวเลขบนกระดาน 3x3

    State:
        Tuple ของตัวเลข 9 ตัว โดยเรียงจากซ้ายไปขวา, บนลงล่าง (0 คือช่องว่าง)
        ตัวอย่าง: (1, 2, 3, 4, 5, 6, 7, 8, 0)

    Action:
        'UP', 'DOWN', 'LEFT', 'RIGHT' คือทิศทางการเลื่อนของ "ช่องว่าง"
    """
    def __init__(self, initial: PuzzleState, goal: PuzzleState = (1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """
        กำหนดสถานะเริ่มต้นและสถานะเป้าหมายของปัญหา

        Args:
            initial (PuzzleState): สถานะเริ่มต้นของบอร์ด
            goal (PuzzleState): สถานะเป้าหมายที่ต้องการไปถึง
        """
        self.initial = initial
        self.goal = goal

        # สร้าง dictionary สำหรับเก็บตำแหน่งเป้าหมายของแต่ละตัวเลข (tile)
        # เพื่อให้สามารถดึงตำแหน่งที่ถูกต้องของตัวเลขแต่ละตัวในสถานะเป้าหมาย (goal state) ได้อย่างรวดเร็ว
        # โดยไม่ต้องใช้ .index() ค้นหาทุกครั้งที่คำนวณ heuristic ซึ่งช่วยเพิ่มประสิทธิภาพ
        # ตัวอย่าง: ถ้า goal=(1,2,3,4,5,6,7,8,0), _goal_positions จะเป็น {1:0, 2:1, ..., 8:7, 0:8}
        self._goal_positions = {tile: i for i, tile in enumerate(self.goal)}

    def actions(self, state: PuzzleState) -> List[PuzzleAction]:
        """
        คืนค่า Action ที่สามารถทำได้จาก State ปัจจุบัน
        โดยตรวจสอบว่าช่องว่าง (0) สามารถเลื่อนไปในทิศทางใดได้บ้าง
        """
        possible_actions: List[PuzzleAction] = []
        # หาตำแหน่ง (index) ของช่องว่าง
        blank_index = state.index(0)
        # แปลง index เป็นพิกัด (แถว, คอลัมน์) เพื่อให้ง่ายต่อการตรวจสอบ
        row, col = divmod(blank_index, 3)

        # ตรวจสอบเงื่อนไขการเลื่อนในแต่ละทิศทาง
        if row > 0: possible_actions.append('UP')    # เลื่อนขึ้นได้ถ้าไม่ได้อยู่แถวบนสุด
        if row < 2: possible_actions.append('DOWN')  # เลื่อนลงได้ถ้าไม่ได้อยู่แถวล่างสุด
        if col > 0: possible_actions.append('LEFT')  # เลื่อนซ้ายได้ถ้าไม่ได้อยู่คอลัมน์ซ้ายสุด
        if col < 2: possible_actions.append('RIGHT') # เลื่อนขวาได้ถ้าไม่ได้อยู่คอลัมน์ขวาสุด

        return possible_actions

    def result(self, state: PuzzleState, action: PuzzleAction) -> PuzzleState:
        """
        คืนค่า State ใหม่หลังจากทำ Action ที่กำหนด
        โดยการสลับตำแหน่งของช่องว่างกับตัวเลขในทิศทางของ action
        """
        blank_index = state.index(0)
        new_state = list(state) # แปลง tuple เป็น list เพื่อให้สามารถแก้ไขค่าได้

        target_index = -1
        # คำนวณ index ของตัวเลขที่จะสลับกับช่องว่าง
        if action == 'UP': target_index = blank_index - 3
        elif action == 'DOWN': target_index = blank_index + 3
        elif action == 'LEFT': target_index = blank_index - 1
        elif action == 'RIGHT': target_index = blank_index + 1

        # สลับตำแหน่งช่องว่างกับตัวเลขเป้าหมาย
        new_state[blank_index], new_state[target_index] = new_state[target_index], new_state[blank_index]

        return tuple(new_state) # แปลง list กลับเป็น tuple เพื่อให้เป็น state ที่ถูกต้อง

    def heuristic(self, state: PuzzleState, goal: PuzzleState) -> float:
        """
        คำนวณฮิวริสติกด้วย Manhattan Distance
        ซึ่งคือผลรวมของระยะทางที่แต่ละตัวเลขต้องย้ายในแนวตั้งและแนวนอนเพื่อไปยังตำแหน่งเป้าหมาย
        ฮิวริสติกนี้เป็น admissible (ไม่ประเมินค่าสูงกว่าความเป็นจริง) สำหรับปัญหา 8-Puzzle
        """
        distance = 0
        # วนลูปเพื่อตรวจสอบตัวเลขทุกตัวในสถานะปัจจุบัน
        for i, tile in enumerate(state):
            # ไม่ต้องคำนวณระยะทางของช่องว่าง (0)
            if tile != 0:
                # หาตำแหน่ง (แถว, คอลัมน์) ปัจจุบันของตัวเลข
                current_row, current_col = divmod(i, 3)

                # ดึงตำแหน่งเป้าหมายของตัวเลขนี้จาก dictionary ที่คำนวณไว้ล่วงหน้าเพื่อความเร็ว
                goal_index = self._goal_positions[tile]

                # หาตำแหน่ง (แถว, คอลัมน์) เป้าหมายของตัวเลข
                goal_row, goal_col = divmod(goal_index, 3)

                # คำนวณ Manhattan Distance สำหรับตัวเลขนี้และบวกเข้ากับผลรวม
                # abs(current_row - goal_row) คือระยะทางในแนวตั้ง
                # abs(current_col - goal_col) คือระยะทางในแนวนอน
                distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return distance

    def view_state(self, state: PuzzleState):
        """สำหรับแสดงผลบอร์ด 3x3 ในรูปแบบที่อ่านง่าย"""
        print("-" * 11)
        for i in range(0, 9, 3):
            # เปลี่ยนเลข 0 เป็น '_' เพื่อความสวยงาม
            row = [str(x) if x != 0 else '_' for x in state[i:i+3]]
            print(f"| {' | '.join(row)} |")
        print("-" * 11)

# ส่วนของการเรียกใช้งาน

In [3]:
def print_puzzle_solution(problem: EightPuzzleProblem, solution_node: Optional[Node]):
    """ฟังก์ชันสำหรับแสดงผลขั้นตอนการแก้ปัญหา 8-Puzzle อย่างละเอียด"""
    if solution_node:
        path = solution_node.path()
        print(f"พบคำตอบแล้ว! ใช้ทั้งหมด {len(path) - 1} ขั้นตอน")
        print(f"Path cost (g(n)) = {solution_node.path_cost}")
        print("=" * 25)
        for i, node in enumerate(path):
            if i == 0:
                print(f"Step {i}: (สถานะเริ่มต้น)")
            else:
                print(f"Step {i}: (Action: {node.action})")

            problem.view_state(node.state)
            # print(f"  g(n)={node.path_cost}, h(n)={problem.heuristic(node.state, problem.goal)}") # Optional: แสดงค่า g(n), h(n)
            print() # เว้นบรรทัด
    else:
        print("ไม่พบคำตอบสำหรับปัญหานี้ (อาจเป็นสถานะที่แก้ไม่ได้)")

# กำหนด State เริ่มต้นและเป้าหมาย
# State นี้สามารถแก้ไขได้ใน 4 ขั้นตอน: UP, UP, LEFT, DOWN
initial_state: PuzzleState = (8, 2, 7, 0, 4, 6, 3, 5, 1)
goal_state: PuzzleState =    (1, 2, 3, 4, 5, 6, 7, 8, 0)

# สร้าง instance ของปัญหา
puzzle_problem = EightPuzzleProblem(initial=initial_state, goal=goal_state)

# เรียกใช้ A* Search เพื่อแก้ปัญหา
# ฟังก์ชัน astar_search จะคืนค่าเป็น Node object ที่เก็บสถานะเป้าหมายและเส้นทางทั้งหมด
# หรือคืนค่าเป็น None หากไม่พบคำตอบ
print("="*10, "กำลังค้นหาด้วย A* Search", "="*10)
solution_node = astar_search(puzzle_problem, puzzle_problem.initial, puzzle_problem.goal)
print("="*40)
print()


# แสดงผลลัพธ์
print_puzzle_solution(puzzle_problem, solution_node)

========== กำลังค้นหาด้วย A* Search ==========

พบคำตอบแล้ว! ใช้ทั้งหมด 29 ขั้นตอน
Path cost (g(n)) = 29.0
Step 0: (สถานะเริ่มต้น)
-----------
| 8 | 2 | 7 |
| _ | 4 | 6 |
| 3 | 5 | 1 |
-----------

Step 1: (Action: RIGHT)
-----------
| 8 | 2 | 7 |
| 4 | _ | 6 |
| 3 | 5 | 1 |
-----------

Step 2: (Action: DOWN)
-----------
| 8 | 2 | 7 |
| 4 | 5 | 6 |
| 3 | _ | 1 |
-----------

Step 3: (Action: LEFT)
-----------
| 8 | 2 | 7 |
| 4 | 5 | 6 |
| _ | 3 | 1 |
-----------

Step 4: (Action: UP)
-----------
| 8 | 2 | 7 |
| _ | 5 | 6 |
| 4 | 3 | 1 |
-----------

Step 5: (Action: RIGHT)
-----------
| 8 | 2 | 7 |
| 5 | _ | 6 |
| 4 | 3 | 1 |
-----------

Step 6: (Action: RIGHT)
-----------
| 8 | 2 | 7 |
| 5 | 6 | _ |
| 4 | 3 | 1 |
-----------

Step 7: (Action: DOWN)
-----------
| 8 | 2 | 7 |
| 5 | 6 | 1 |
| 4 | 3 | _ |
-----------

Step 8: (Action: LEFT)
-----------
| 8 | 2 | 7 |
| 5 | 6 | 1 |
| 4 | _ | 3 |
-----------

Step 9: (Action: UP)
-----------
| 8 | 2 | 7 |
| 5 | _ | 1 |
| 4 | 6 | 3 |
------